# Circular inhomogeneity — Shear

The goal of this notebook is to derive the closed form solution for a sheare composite disk in 2d elasticity.

We consider a circular inclusion (radius: $a$, Lamé coefficients: $\lambda_{\mathrm i}$, $\mu_{\mathrm i}$) embedded in a homogeneous, circular matrix (radius: $R$, Lamé coefficients: $\lambda_{\mathrm m}$, $\mu_{\mathrm m}$). Both disks are concentric.

The boundary of the matrix is subjected to the prescribed displacement

\begin{equation*}
u_x = y\quad\text{and}\quad u_y=x.
\end{equation*}

These boundary conditions are known as “kinematically uniform boundary conditions”, the general form of which is

\begin{equation*}
\mathbf{u} = \overline{\boldsymbol\varepsilon}\cdot\mathbf{x},
\end{equation*}

where the *macroscopic strain* $\overline{\boldsymbol\varepsilon}$ is in the present case a unit shear

\begin{equation*}
\overline{\boldsymbol\varepsilon}=\mathbf{e}_x\otimes\mathbf{e}_y+\mathbf{e}_y\otimes\mathbf{e}_x.
\end{equation*}

We seek the solution to this problem in polar coordinates, assuming a separated form

\begin{equation*}
u_r = F(r)\Phi(\theta)\quad\text{and}\quad u_\theta = G(r)\Psi(\theta).
\end{equation*}

We first identify the functions $\Phi$ and $\Psi$ from the boundary conditions.

In [1]:
import sympy

In [2]:
sympy.init_printing(use_latex="mathjax")

## Boundary conditions in polar coordinates

We first expand the vectors of the cartesian basis in the polar basis.

In [3]:
r, θ = sympy.symbols("r theta")

In [4]:
cos_θ = sympy.cos(θ)
sin_θ = sympy.sin(θ)

In [5]:
e_x = sympy.Matrix([cos_θ, -sin_θ])
e_y = sympy.Matrix([sin_θ,  cos_θ])

Recalling that in matrix form, $\mathbf{u}\otimes\mathbf{v}$ is computed as $\mathsf{u}\cdot\mathsf{v}^{\mathsf T}$, we then express the macroscopic strain in the polar basis (note that the `@` operator denotes the matrix-matrix product in python).

In [6]:
eps_macro = e_x @ e_y.T + e_y @ e_x.T
eps_macro = eps_macro.applyfunc(sympy.simplify)
eps_macro

⎡sin(2⋅θ)  cos(2⋅θ) ⎤
⎢                   ⎥
⎣cos(2⋅θ)  -sin(2⋅θ)⎦

In polar coordinates, we have on the boundary $\mathbf x=R\,\mathbf e_r$. We introduce the dimensionless ratio $\gamma$, such that $R=\gamma a$ ($a$: radius of the inhomogeneity).

In [7]:
a = sympy.Symbol("a")
γ = sympy.Symbol("gamma")
R = γ*a

In [8]:
x = sympy.Matrix([R, 0])

From which we deduce the prescribed displacement in polar coordinates

In [9]:
u_prescribed = eps_macro @ x
u_prescribed

⎡a⋅γ⋅sin(2⋅θ)⎤
⎢            ⎥
⎣a⋅γ⋅cos(2⋅θ)⎦

This suggests the following functions $\Phi$ and $\Psi$

In [10]:
Φ, Ψ = u_prescribed/R

In [11]:
Φ

sin(2⋅θ)

In [12]:
Ψ

cos(2⋅θ)

## General form of $F$ and $G$

It will be convenient to introduce the reduced radius $\rho=r/a$.

In [13]:
ρ, E, ν = sympy.symbols("rho E nu")
μ = E/2/(1+ν)
λ = 2*μ*ν/(1-2*ν)

In [14]:
F = sympy.Function("F")(ρ)
G = sympy.Function("G")(ρ)

In [15]:
u = sympy.Matrix([F*Φ, G*Ψ])
u

⎡F(ρ)⋅sin(2⋅θ)⎤
⎢             ⎥
⎣G(ρ)⋅cos(2⋅θ)⎦

In [16]:
u_r, u_θ = u
grad_u = sympy.Matrix([[u_r.diff(ρ), (u_r.diff(θ)-u_θ)/ρ],
                       [u_θ.diff(ρ), (u_θ.diff(θ)+u_r)/ρ]])/a
grad_u

⎡         d                                        ⎤
⎢sin(2⋅θ)⋅──(F(ρ))                                 ⎥
⎢         dρ        2⋅F(ρ)⋅cos(2⋅θ) - G(ρ)⋅cos(2⋅θ)⎥
⎢─────────────────  ───────────────────────────────⎥
⎢        a                        a⋅ρ              ⎥
⎢                                                  ⎥
⎢         d                                        ⎥
⎢cos(2⋅θ)⋅──(G(ρ))                                 ⎥
⎢         dρ        F(ρ)⋅sin(2⋅θ) - 2⋅G(ρ)⋅sin(2⋅θ)⎥
⎢─────────────────  ───────────────────────────────⎥
⎣        a                        a⋅ρ              ⎦

In [17]:
ε = (grad_u+grad_u.T)/2
ε = ε.applyfunc(sympy.expand)
ε

⎡                         d                                  d                
⎢                sin(2⋅θ)⋅──(F(ρ))                  cos(2⋅θ)⋅──(G(ρ))         
⎢                         dρ                                 dρ         F(ρ)⋅c
⎢                ─────────────────                  ───────────────── + ──────
⎢                        a                                 2⋅a               a
⎢                                                                             
⎢         d                                                                   
⎢cos(2⋅θ)⋅──(G(ρ))                                                            
⎢         dρ         F(ρ)⋅cos(2⋅θ)   G(ρ)⋅cos(2⋅θ)           F(ρ)⋅sin(2⋅θ)   2
⎢───────────────── + ───────────── - ─────────────           ───────────── - ─
⎣       2⋅a               a⋅ρ            2⋅a⋅ρ                    a⋅ρ         

                       ⎤
                       ⎥
os(2⋅θ)   G(ρ)⋅cos(2⋅θ)⎥
─────── - ─────────────⎥
⋅ρ            2⋅a⋅ρ    ⎥
     

In [18]:
I = sympy.eye(2)
σ = λ*ε.trace()*I+2*μ*ε
σ = σ.applyfunc(sympy.expand)
σ

⎡                                                                   d         
⎢                                                      E⋅ν⋅sin(2⋅θ)⋅──(F(ρ))  
⎢   E⋅ν⋅F(ρ)⋅sin(2⋅θ)         2⋅E⋅ν⋅G(ρ)⋅sin(2⋅θ)                   dρ        
⎢──────────────────────── - ──────────────────────── + ───────────────────── +
⎢       2                          2                            2             
⎢- 2⋅a⋅ν ⋅ρ - a⋅ν⋅ρ + a⋅ρ   - 2⋅a⋅ν ⋅ρ - a⋅ν⋅ρ + a⋅ρ     - 2⋅a⋅ν  - a⋅ν + a   
⎢                                                                             
⎢                                                                     d       
⎢                                                          E⋅cos(2⋅θ)⋅──(G(ρ))
⎢                      E⋅G(ρ)⋅cos(2⋅θ)   E⋅F(ρ)⋅cos(2⋅θ)              dρ      
⎢                    - ─────────────── + ─────────────── + ───────────────────
⎢                      2⋅a⋅ν⋅ρ + 2⋅a⋅ρ     a⋅ν⋅ρ + a⋅ρ         2⋅a⋅ν + 2⋅a    
⎣                                                   

In [19]:
σ_rr = σ[0, 0]
σ_rθ = σ[0, 1]
σ_θθ = σ[1, 1]

In [20]:
div_σ = (sympy.Matrix([σ_rr.diff(ρ)+(σ_rθ.diff(θ)+σ_rr-σ_θθ)/ρ,
                       σ_rθ.diff(ρ)+(σ_θθ.diff(θ)+2*σ_rθ)/ρ])/a).applyfunc(sympy.factor)

In [21]:
div_σ

⎡       ⎛       2                                                   2         
⎢       ⎜   2  d              d                                 2  d          
⎢     E⋅⎜ν⋅ρ ⋅───(F(ρ)) + ν⋅ρ⋅──(F(ρ)) - 5⋅ν⋅F(ρ) + 4⋅ν⋅G(ρ) - ρ ⋅───(F(ρ)) - 
⎢       ⎜       2             dρ                                    2         
⎢       ⎝     dρ                                                  dρ          
⎢     ────────────────────────────────────────────────────────────────────────
⎢                                                        2  2                 
⎢                                                       a ⋅ρ ⋅(ν + 1)⋅(2⋅ν - 1
⎢                                                                             
⎢   ⎛           2                                                      2      
⎢   ⎜       2  d                d                                  2  d       
⎢-E⋅⎜- 2⋅ν⋅ρ ⋅───(G(ρ)) - 2⋅ν⋅ρ⋅──(G(ρ)) - 8⋅ν⋅F(ρ) + 10⋅ν⋅G(ρ) + ρ ⋅───(G(ρ))
⎢   ⎜           2               dρ                  

From the equilibrium equations, we get the following equations

In [22]:
eq1, eq2 = ((1-2*ν)*(1+ν)*ρ**2*a**2/E*div_σ).applyfunc(sympy.factor)
eq1 /= sympy.sin(2*θ)
eq2 /= sympy.cos(2*θ)
eq1 = eq1.expand()
eq2 = eq2.expand()

In [23]:
eq1

         2                                                   2                
     2  d              d                                 2  d            d    
- ν⋅ρ ⋅───(F(ρ)) - ν⋅ρ⋅──(F(ρ)) + 5⋅ν⋅F(ρ) - 4⋅ν⋅G(ρ) + ρ ⋅───(F(ρ)) + ρ⋅──(F(
         2             dρ                                    2           dρ   
       dρ                                                  dρ                 

                                  
        d                         
ρ)) - ρ⋅──(G(ρ)) - 3⋅F(ρ) + 3⋅G(ρ)
        dρ                        
                                  

and

In [24]:
eq2

                                                             2                
                                                         2  d                 
                                                        ρ ⋅───(G(ρ))          
         2                                                   2                
     2  d              d                                   dρ            d    
- ν⋅ρ ⋅───(G(ρ)) - ν⋅ρ⋅──(G(ρ)) - 4⋅ν⋅F(ρ) + 5⋅ν⋅G(ρ) + ──────────── + ρ⋅──(F(
         2             dρ                                    2           dρ   
       dρ                                                                     

                                  
                                  
        d                         
      ρ⋅──(G(ρ))                  
        dρ                  9⋅G(ρ)
ρ)) + ────────── + 3⋅F(ρ) - ──────
          2                   2   
                                  

We seek solutions of the form

\begin{equation*}
F(\rho)=A\rho^n\quad\text{and}\quad G(\rho)=B\rho^n
\end{equation*}

In [25]:
A, B, n = sympy.symbols("A B n")
substitutions = {F: A*ρ**n, G: B*ρ**n}

Which delivers the two equations

In [26]:
eq3 = eq1.subs(substitutions).doit().factor()/ρ**n
eq3

     2        2                                  
- A⋅n ⋅ν + A⋅n  + 5⋅A⋅ν - 3⋅A - B⋅n - 4⋅B⋅ν + 3⋅B

and

In [27]:
eq4 = eq2.subs(substitutions).doit().factor()/ρ**n
eq4

                                2              
                       2     B⋅n            9⋅B
A⋅n - 4⋅A⋅ν + 3⋅A - B⋅n ⋅ν + ──── + 5⋅B⋅ν - ───
                              2              2 

Or, in matrix form: $\mathsf{M}\cdot[A, B]^{\mathsf T}=0$, where $\mathsf M$ is the following matrix

In [28]:
M, _ = sympy.linear_eq_to_matrix([eq3, eq4], [A, B])
M

⎡   2      2                                 ⎤
⎢- n ⋅ν + n  + 5⋅ν - 3      -n - 4⋅ν + 3     ⎥
⎢                                            ⎥
⎢                                 2          ⎥
⎢                          2     n          9⎥
⎢     n - 4⋅ν + 3       - n ⋅ν + ── + 5⋅ν - ─⎥
⎣                                2          2⎦

For the resulting system of equations to have a non-trivial solution $A=0$ and $B=0$, $\det\mathsf M$ must be zero.

In [29]:
det_M = M.det().factor()
det_M

(n - 3)⋅(n - 1)⋅(n + 1)⋅(n + 3)⋅(ν - 1)⋅(2⋅ν - 1)
─────────────────────────────────────────────────
                        2                        

In [30]:
n_sol = sympy.solve(det_M, n)
n_sol

[-3, -1, 1, 3]

For each value of $n$, we obtain a linear relationship between $A$ and $B$

In [31]:
A_sol = {}
B_sol = {}
F_sol = 0
G_sol = 0
for n_i in n_sol:
    A_n = sympy.Symbol(f"A_{n_i}")
    B_n = sympy.solve(eq3.subs({n: n_i, A: A_n}), B)[0]
    A_sol[n_i] = A_n
    B_sol[n_i] = B_n
    F_sol += A_n*ρ**n_i
    G_sol += B_n*ρ**n_i

And we have the general expression of $F$ and $G$

In [32]:
F_sol

A₋₁   A₋₃              3
─── + ─── + A₁⋅ρ + A₃⋅ρ 
 ρ      3               
       ρ                

In [33]:
G_sol

A₋₁⋅(2⋅ν - 1)   A₋₃           3 ⎛      3⋅A₃⎞
───────────── - ─── + A₁⋅ρ + ρ ⋅⎜-A₃ + ────⎟
 2⋅ρ⋅(ν - 1)      3             ⎝      2⋅ν ⎠
                 ρ                          

## Construction of the general solution in the matrix

In the matrix, the Young modulus and Poisson ratio are $E_{\mathrm m}$ and $\nu_{\mathrm m}$. In the matrix, all terms of $F$ and $G$ must be kept.

In [34]:
E_m = sympy.Symbol(r"E_{\mathrm m}")
ν_m = sympy.Symbol(r"\nu_{\mathrm m}")

In [35]:
substitutions = {F: F_sol, G: G_sol, E: E_m, ν: ν_m}
my_subs = lambda expr: expr.subs(substitutions).doit().factor()

In [36]:
u_m, ε_m, σ_m = (expr.applyfunc(my_subs) for expr in (u, ε, σ))

In [37]:
u_m

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢⎛                     2  2                        2                        2 
⎢⎝2⋅A₋₁⋅\nu_{\mathrm m} ⋅ρ  - A₋₁⋅\nu_{\mathrm m}⋅ρ  - 2⋅A₋₃⋅\nu_{\mathrm m}  
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎣                                                                             

                ⎛     2             4       6⎞                                
                ⎝A₋₁⋅ρ  + A₋₃ + A₁⋅ρ  + A₃⋅ρ ⎠⋅sin(

In [38]:
ε_m

⎡                                                                ⎛       2    
⎢                                                                ⎝- A₋₁⋅ρ  - 3
⎢                                                                ─────────────
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢⎛                       2                        2                           
⎢⎝- A₋₁⋅\nu_{\mathrm m}⋅ρ  + 6⋅A₋₃⋅\nu_{\mathrm m}  - 6⋅A₋₃⋅\nu_{\mathrm m} + 
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎣                                                                 2⋅\nu_{\math

           4         6⎞                                                       
⋅A₋₃ + A₁⋅ρ  + 3⋅A₃⋅ρ ⎠⋅sin(2⋅θ)                   

In [39]:
σ_m

⎡                                                            ⎛     2          
⎢                                              E_{\mathrm m}⋅⎝A₋₁⋅ρ  - 3⋅A₋₃⋅\
⎢                                              ───────────────────────────────
⎢                                                                         4   
⎢                                                                      a⋅ρ ⋅(\
⎢                                                                             
⎢              ⎛                       2                        2             
⎢E_{\mathrm m}⋅⎝- A₋₁⋅\nu_{\mathrm m}⋅ρ  + 6⋅A₋₃⋅\nu_{\mathrm m}  - 6⋅A₋₃⋅\nu_
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎣                                                             2⋅\nu_{\mathrm m

                                             4       4⎞                       
nu_{\mathrm m} + 3⋅A₋₃ + A₁⋅\nu_{\mathrm m}⋅ρ  - A₁

We must express that the boundary conditions (at $r=R$) are satisfied, which delivers two equations.

In [40]:
eq5, eq6 = (u_m.subs(ρ, γ)-u_prescribed).applyfunc(sympy.factor)
eq5 *= 2*γ**3/sympy.sin(2*θ)
eq6 *= 2*γ**3*ν_m*(1-ν_m)/sympy.cos(2*θ)
eq6 = eq6.factor()

In [41]:
eq5

       2                 4         6        4
2⋅A₋₁⋅γ  + 2⋅A₋₃ + 2⋅A₁⋅γ  + 2⋅A₃⋅γ  - 2⋅a⋅γ 

In [42]:
eq6

                       2  2                        2                        2 
- 2⋅A₋₁⋅\nu_{\mathrm m} ⋅γ  + A₋₁⋅\nu_{\mathrm m}⋅γ  + 2⋅A₋₃⋅\nu_{\mathrm m}  

                                              2  4                         4  
- 2⋅A₋₃⋅\nu_{\mathrm m} - 2⋅A₁⋅\nu_{\mathrm m} ⋅γ  + 2⋅A₁⋅\nu_{\mathrm m}⋅γ  +

                     2  6                         6         6                 
 2⋅A₃⋅\nu_{\mathrm m} ⋅γ  - 5⋅A₃⋅\nu_{\mathrm m}⋅γ  + 3⋅A₃⋅γ  + 2⋅\nu_{\mathrm

   2    4                        4
 m} ⋅a⋅γ  - 2⋅\nu_{\mathrm m}⋅a⋅γ 

## Construction of the solution inside the inhomogeneity

For $r\leq a$ ($\rho\leq 1$), we must eliminate the singular terms. In the inhomogeneity, the Young modulus and Poisson ratio are $E_{\mathrm i}=\chi E_{\mathrm m}$ ($\chi$: dimensionless ratio, elastic contrast) and $\nu_{\mathrm i}$.

In [43]:
χ = sympy.Symbol("chi")
ν_i = sympy.Symbol(r"\nu_{\mathrm i}")

In [44]:
unknowns = list(A_sol.values())
substitutions = {E_m: χ*E_m, ν_m: ν_i}

for n_, A_n in A_sol.items():
    if n_ <= 0:
        substitutions[A_n] = 0
    else:
        C_n = sympy.Symbol(f"C_{n_}")
        unknowns.append(C_n)
        substitutions[A_n] = C_n
display(substitutions)
display(unknowns)

{A₋₁: 0, A₋₃: 0, A₁: C₁, A₃: C₃, E_{\mathrm m}: E_{\mathrm m}⋅χ, \nu_{\mathrm 
m}: \nu_{\mathrm i}}

[A₋₃, A₋₁, A₁, A₃, C₁, C₃]

In [45]:
my_subs = lambda expr: expr.subs(substitutions).factor()

u_i, ε_i, σ_i = (expr.applyfunc(my_subs) for expr in (u_m, ε_m, σ_m))

In [46]:
u_i

⎡                         ⎛         2⎞                                ⎤
⎢                       ρ⋅⎝C₁ + C₃⋅ρ ⎠⋅sin(2⋅θ)                       ⎥
⎢                                                                     ⎥
⎢  ⎛                                             2         2⎞         ⎥
⎢ρ⋅⎝2⋅C₁⋅\nu_{\mathrm i} - 2⋅C₃⋅\nu_{\mathrm i}⋅ρ  + 3⋅C₃⋅ρ ⎠⋅cos(2⋅θ)⎥
⎢─────────────────────────────────────────────────────────────────────⎥
⎣                          2⋅\nu_{\mathrm i}                          ⎦

In [47]:
ε_i

⎡         ⎛           2⎞                                 ⎛                    
⎢         ⎝C₁ + 3⋅C₃⋅ρ ⎠⋅sin(2⋅θ)                        ⎝2⋅C₁⋅\nu_{\mathrm i}
⎢         ───────────────────────                        ─────────────────────
⎢                    a                                              2⋅\nu_{\ma
⎢                                                                             
⎢⎛                             2⎞            ⎛                                
⎢⎝2⋅C₁⋅\nu_{\mathrm i} + 3⋅C₃⋅ρ ⎠⋅cos(2⋅θ)  -⎝C₁⋅\nu_{\mathrm i} - 3⋅C₃⋅\nu_{\
⎢─────────────────────────────────────────  ──────────────────────────────────
⎣           2⋅\nu_{\mathrm i}⋅a                                      \nu_{\mat

         2⎞                      ⎤
 + 3⋅C₃⋅ρ ⎠⋅cos(2⋅θ)             ⎥
────────────────────             ⎥
thrm i}⋅a                        ⎥
                                 ⎥
           2         2⎞          ⎥
mathrm i}⋅ρ  + 3⋅C₃⋅ρ ⎠⋅sin(2⋅θ) ⎥
─────────────────────────────────⎥
hrm i}⋅a

In [48]:
σ_i

⎡                                                                           ⎛ 
⎢               C₁⋅E_{\mathrm m}⋅χ⋅sin(2⋅θ)                 E_{\mathrm m}⋅χ⋅⎝2
⎢               ───────────────────────────                 ──────────────────
⎢                 a⋅(\nu_{\mathrm i} + 1)                           2⋅\nu_{\ma
⎢                                                                             
⎢                ⎛                             2⎞                            ⎛
⎢E_{\mathrm m}⋅χ⋅⎝2⋅C₁⋅\nu_{\mathrm i} + 3⋅C₃⋅ρ ⎠⋅cos(2⋅θ)  -E_{\mathrm m}⋅χ⋅⎝
⎢─────────────────────────────────────────────────────────  ──────────────────
⎣        2⋅\nu_{\mathrm i}⋅a⋅(\nu_{\mathrm i} + 1)                   \nu_{\mat

                            2⎞         ⎤
⋅C₁⋅\nu_{\mathrm i} + 3⋅C₃⋅ρ ⎠⋅cos(2⋅θ)⎥
───────────────────────────────────────⎥
thrm i}⋅a⋅(\nu_{\mathrm i} + 1)        ⎥
                                       ⎥
                           2⎞          ⎥
C₁⋅\nu_{\mathrm i} + 3⋅C₃⋅ρ ⎠⋅sin(2⋅θ) ⎥
─

## Continuity at the interface

We first express that the displacements are continuous at $r=a$, which delivers two equations

In [49]:
eq7, eq8 = (u_m-u_i).subs(ρ, 1).applyfunc(sympy.factor)
eq7 /= sympy.sin(2*θ)
eq8 *= 2*ν_i*ν_m*(1-ν_m)/sympy.cos(2*θ)
eq8 = eq8.factor()

In [50]:
eq7

A₋₁ + A₋₃ + A₁ + A₃ - C₁ - C₃

In [51]:
eq8

                                       2                                      
- 2⋅A₋₁⋅\nu_{\mathrm i}⋅\nu_{\mathrm m}  + A₋₁⋅\nu_{\mathrm i}⋅\nu_{\mathrm m}

                                        2                                     
 + 2⋅A₋₃⋅\nu_{\mathrm i}⋅\nu_{\mathrm m}  - 2⋅A₋₃⋅\nu_{\mathrm i}⋅\nu_{\mathrm

                                          2                                   
 m} - 2⋅A₁⋅\nu_{\mathrm i}⋅\nu_{\mathrm m}  + 2⋅A₁⋅\nu_{\mathrm i}⋅\nu_{\mathr

                                           2                                  
m m} + 2⋅A₃⋅\nu_{\mathrm i}⋅\nu_{\mathrm m}  - 5⋅A₃⋅\nu_{\mathrm i}⋅\nu_{\math

                                                                   2          
rm m} + 3⋅A₃⋅\nu_{\mathrm i} + 2⋅C₁⋅\nu_{\mathrm i}⋅\nu_{\mathrm m}  - 2⋅C₁⋅\n

                                                                    2         
u_{\mathrm i}⋅\nu_{\mathrm m} - 2⋅C₃⋅\nu_{\mathrm i}⋅\nu_{\mathrm m}  + 2⋅C₃⋅\

                                              

Finally, the traction is continuous

In [52]:
eq9, eq10 = (σ_m-σ_i).col(0).subs(ρ, 1).applyfunc(sympy.factor)
eq9 *= a*(1+ν_i)*(1-ν_m**2)/E_m/sympy.sin(2*θ)
eq10 *= 2*a*ν_i*ν_m*(1+ν_i)*(1-ν_m**2)/E_m/sympy.cos(2*θ)
eq9 = eq9.factor()
eq10 = eq10.factor()

In [53]:
eq9

                                                                              
-A₋₁⋅\nu_{\mathrm i} - A₋₁ + 3⋅A₋₃⋅\nu_{\mathrm i}⋅\nu_{\mathrm m} - 3⋅A₋₃⋅\nu

                                                                              
_{\mathrm i} + 3⋅A₋₃⋅\nu_{\mathrm m} - 3⋅A₋₃ - A₁⋅\nu_{\mathrm i}⋅\nu_{\mathrm

                                                                       2      
 m} + A₁⋅\nu_{\mathrm i} - A₁⋅\nu_{\mathrm m} + A₁ + C₁⋅\nu_{\mathrm m} ⋅χ - C

   
₁⋅χ

In [54]:
eq10

                   2                                                          
A₋₁⋅\nu_{\mathrm i} ⋅\nu_{\mathrm m} + A₋₁⋅\nu_{\mathrm i}⋅\nu_{\mathrm m} - 6

                    2                2                        2               
⋅A₋₃⋅\nu_{\mathrm i} ⋅\nu_{\mathrm m}  + 6⋅A₋₃⋅\nu_{\mathrm i} ⋅\nu_{\mathrm m

                                         2                                    
} - 6⋅A₋₃⋅\nu_{\mathrm i}⋅\nu_{\mathrm m}  + 6⋅A₋₃⋅\nu_{\mathrm i}⋅\nu_{\mathr

                           2                2                       2         
m m} - 2⋅A₁⋅\nu_{\mathrm i} ⋅\nu_{\mathrm m}  + 2⋅A₁⋅\nu_{\mathrm i} ⋅\nu_{\ma

                                              2                               
thrm m} - 2⋅A₁⋅\nu_{\mathrm i}⋅\nu_{\mathrm m}  + 2⋅A₁⋅\nu_{\mathrm i}⋅\nu_{\m

                               2                                       2      
athrm m} - 3⋅A₃⋅\nu_{\mathrm i} ⋅\nu_{\mathrm m} + 3⋅A₃⋅\nu_{\mathrm i}  - 3⋅A

                                              

## Computing the solution

We are left with a system of 6 equations for 6 unknowns!

In [55]:
unknowns

[A₋₃, A₋₁, A₁, A₃, C₁, C₃]

In [56]:
lhs, rhs = sympy.linear_eq_to_matrix([eq5, eq6, eq7, eq8, eq9, eq10], unknowns)
lhs = lhs.applyfunc(sympy.factor)
rhs = rhs.applyfunc(sympy.factor)

In [57]:
lhs

⎡                                                                             
⎢                                      2                                      
⎢                                                                             
⎢                                                                             
⎢                   2⋅\nu_{\mathrm m}⋅(\nu_{\mathrm m} - 1)                   
⎢                                                                             
⎢                                      1                                      
⎢                                                                             
⎢           2⋅\nu_{\mathrm i}⋅\nu_{\mathrm m}⋅(\nu_{\mathrm m} - 1)           
⎢                                                                             
⎢                3⋅(\nu_{\mathrm i} + 1)⋅(\nu_{\mathrm m} - 1)                
⎢                                                                             
⎣-6⋅\nu_{\mathrm i}⋅\nu_{\mathrm m}⋅(\nu_{\mathrm i}

In [58]:
rhs

⎡                        4                    ⎤
⎢                   2⋅a⋅γ                     ⎥
⎢                                             ⎥
⎢                      4                      ⎥
⎢-2⋅\nu_{\mathrm m}⋅a⋅γ ⋅(\nu_{\mathrm m} - 1)⎥
⎢                                             ⎥
⎢                      0                      ⎥
⎢                                             ⎥
⎢                      0                      ⎥
⎢                                             ⎥
⎢                      0                      ⎥
⎢                                             ⎥
⎣                      0                      ⎦

We will not compute the analytical solution to the above problem. Rather, we will keep it in a matrix form, and solve the underlying numerical system numerically on demand.

To do so, we use the code generation capabilities of SymPy.

In [59]:
printer = sympy.printing.lambdarepr.NumPyPrinter()
substitutions = {ν_i: sympy.Symbol("nu_i"), ν_m: sympy.Symbol("nu_m")}

In [60]:
printer.doprint(lhs.subs(substitutions))

'numpy.array([[2, 2*gamma**2, 2*gamma**4, 2*gamma**6, 0, 0], [2*nu_m*(nu_m - 1), -gamma**2*nu_m*(2*nu_m - 1), -2*gamma**4*nu_m*(nu_m - 1), gamma**6*(nu_m - 1)*(2*nu_m - 3), 0, 0], [1, 1, 1, 1, -1, -1], [2*nu_i*nu_m*(nu_m - 1), -nu_i*nu_m*(2*nu_m - 1), -2*nu_i*nu_m*(nu_m - 1), nu_i*(nu_m - 1)*(2*nu_m - 3), 2*nu_i*nu_m*(nu_m - 1), -nu_m*(2*nu_i - 3)*(nu_m - 1)], [3*(nu_i + 1)*(nu_m - 1), -nu_i - 1, -(nu_i + 1)*(nu_m - 1), 0, chi*(nu_m - 1)*(nu_m + 1), 0], [-6*nu_i*nu_m*(nu_i + 1)*(nu_m - 1), nu_i*nu_m*(nu_i + 1), -2*nu_i*nu_m*(nu_i + 1)*(nu_m - 1), -3*nu_i*(nu_i + 1)*(nu_m - 1), 2*chi*nu_i*nu_m*(nu_m - 1)*(nu_m + 1), 3*chi*nu_m*(nu_m - 1)*(nu_m + 1)]])'

In [61]:
printer.doprint(rhs.subs(substitutions).T)

'numpy.array([[2*a*gamma**4, -2*a*gamma**4*nu_m*(nu_m - 1), 0, 0, 0, 0]])'